In [24]:
from classes_taskonomy import ImageDataset, calculate_dataset_correlation, correlate_integration_beauty
# modified visualpriors library
from transforms import VisualPrior
from transforms import VisualPriorRepresentation, VisualPriorPredictedLabel
from taskonomy_network import TaskonomyEncoder, TaskonomyDecoder

import sys
import os
import collections
import warnings

import numpy as np 
import pandas as pd
from math import isnan

import matplotlib.pyplot as plt
import torchsummary

from PIL import Image 
from scipy.stats import pearsonr, PearsonRConstantInputWarning
from scipy.io import savemat, loadmat

import torch
import torch.utils.model_zoo # required to load nets 
import torchvision.transforms.functional as TF
from torch import nn 
import torchvision.models
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

In [30]:
from platform import python_version

print(python_version())

3.8.19


# Load images

### Img.open+TF.to_tensor do a absoloute(/255) or Min-Max scale ?

In [32]:
# load all images
img_dir = "./data/stimuli_places1"

dir_img_list = list(f for f in os.listdir(img_dir)
                            if os.path.isfile(os.path.join(img_dir, f)) and f.endswith('.jpg'))

In [39]:
lmin, lmax = [],[]
for image_name in dir_img_list:
    img = TF.to_tensor(Image.open(os.path.join(img_dir, image_name)))
    lmin.append(img.min())
    lmax.append(img.max())

In [46]:
torch.tensor(lmax).min()

tensor(0.7412)

In [48]:
torch.tensor(lmin).max()

tensor(0.2000)

Img.open+TF.to_tensor does a absoloute scale.

### compare matlab image import to Image.open import

In [137]:
img_matlab = torch.tensor(loadmat('1153_unaltered_matlab.mat')['im']).permute([2, 0, 1])

In [138]:
img_pytorch = TF.to_tensor(Image.open('./data/stimuli_places1/Places365_val_00001153.jpg'))

In [139]:
img_matlab.shape

torch.Size([3, 683, 512])

In [140]:
img_pytorch.shape

torch.Size([3, 683, 512])

In [141]:
img_matlab

tensor([[[0.0275, 0.0627, 0.0667,  ..., 0.6078, 0.7412, 0.6549],
         [0.0667, 0.0667, 0.0431,  ..., 0.4706, 0.5529, 0.4275],
         [0.1882, 0.1490, 0.1020,  ..., 0.3294, 0.3725, 0.2549],
         ...,
         [0.0314, 0.0275, 0.0275,  ..., 0.0824, 0.0784, 0.0745],
         [0.0314, 0.0275, 0.0275,  ..., 0.0824, 0.0784, 0.0745],
         [0.0314, 0.0275, 0.0275,  ..., 0.0824, 0.0784, 0.0745]],

        [[0.0157, 0.0549, 0.0588,  ..., 0.4588, 0.6157, 0.5333],
         [0.0588, 0.0588, 0.0353,  ..., 0.3294, 0.4235, 0.3137],
         [0.1922, 0.1529, 0.1059,  ..., 0.2039, 0.2588, 0.1451],
         ...,
         [0.0549, 0.0510, 0.0510,  ..., 0.2235, 0.2196, 0.2157],
         [0.0549, 0.0510, 0.0510,  ..., 0.2235, 0.2196, 0.2157],
         [0.0549, 0.0510, 0.0510,  ..., 0.2235, 0.2196, 0.2157]],

        [[0.0000, 0.0078, 0.0078,  ..., 0.3686, 0.5333, 0.4549],
         [0.0118, 0.0118, 0.0000,  ..., 0.2431, 0.3490, 0.2431],
         [0.1373, 0.0902, 0.0353,  ..., 0.1216, 0.1961, 0.

In [142]:
img_pytorch

tensor([[[0.0275, 0.0627, 0.0667,  ..., 0.6078, 0.7412, 0.6549],
         [0.0588, 0.0667, 0.0431,  ..., 0.4706, 0.5490, 0.4275],
         [0.1882, 0.1490, 0.1020,  ..., 0.3294, 0.3725, 0.2510],
         ...,
         [0.0314, 0.0275, 0.0275,  ..., 0.0824, 0.0784, 0.0745],
         [0.0314, 0.0275, 0.0275,  ..., 0.0824, 0.0784, 0.0745],
         [0.0314, 0.0275, 0.0275,  ..., 0.0824, 0.0784, 0.0745]],

        [[0.0157, 0.0510, 0.0588,  ..., 0.4588, 0.6157, 0.5333],
         [0.0588, 0.0588, 0.0353,  ..., 0.3294, 0.4275, 0.3137],
         [0.1882, 0.1529, 0.1059,  ..., 0.2000, 0.2588, 0.1490],
         ...,
         [0.0549, 0.0510, 0.0510,  ..., 0.2235, 0.2196, 0.2157],
         [0.0549, 0.0510, 0.0510,  ..., 0.2235, 0.2196, 0.2157],
         [0.0549, 0.0510, 0.0510,  ..., 0.2235, 0.2196, 0.2157]],

        [[0.0000, 0.0157, 0.0118,  ..., 0.3686, 0.5333, 0.4549],
         [0.0196, 0.0118, 0.0000,  ..., 0.2431, 0.3490, 0.2431],
         [0.1412, 0.0980, 0.0431,  ..., 0.1255, 0.1961, 0.

In [143]:
(img_matlab-img_pytorch).abs().sum() / (256*256*3)

tensor(0.0103)

In [47]:
loadmat("data/stimuli_places1_resized/Places365_val_00001153.mat")['im']
import subprocess
subprocess.call("curl -O https://raw.githubusercontent.com/StanfordVL/taskonomy/master/taskbank/assets/test.png", shell=True)
img = Image.open('./data/stimuli_places1/Places365_val_00001153.jpg')
TF.to_tensor(img)

img = TF.to_tensor(TF.resize(img, 256)) * 2 - 1 # resize
img = img.unsqueeze_(0)

# load model
task = 'autoencoding'

VisualPriorRepresentation._load_unloaded_nets([task]) # preload nets
net = VisualPriorRepresentation.feature_task_to_net[task] # loads encoder only for representations
torchsummary.summary(net, (3, 256, 256))

array([[[ 24.440285 ,  22.75106  ,  11.813852 ],
        [ 28.500277 ,  27.750185 ,  10.516175 ],
        [ 46.8996   ,  46.354485 ,  20.093699 ],
        ...,
        [153.51295  , 112.667366 ,  73.45321  ],
        [114.17797  ,  74.978455 ,  48.79689  ],
        [128.2486   ,  97.46462  ,  79.03427  ]],

       [[ 58.05664  ,  61.188354 ,  38.276947 ],
        [ 54.278408 ,  57.572754 ,  31.45102  ],
        [ 52.65642  ,  55.115265 ,  25.720566 ],
        ...,
        [120.24372  ,  86.13118  ,  58.90142  ],
        [ 59.342533 ,  29.822533 ,  15.940515 ],
        [ 42.107304 ,  22.485958 ,  14.650325 ]],

       [[ 36.602886 ,  44.835506 ,  11.999634 ],
        [ 41.501434 ,  49.640537 ,  15.932427 ],
        [ 40.36918  ,  46.6992   ,  15.063794 ],
        ...,
        [149.50587  , 125.88573  , 112.681854 ],
        [ 44.91447  ,  27.869555 ,  25.124537 ],
        [  7.946582 ,   2.764554 ,   4.4361887]],

       ...,

       [[  5.589234 ,  10.654429 ,   4.7645097],
        [  

In [ ]:
# rescale to [-1, 1]

# Build activation extractor

## Load model

In [2]:
task = 'autoencoding'
VisualPriorRepresentation._load_unloaded_nets([task])
net = VisualPriorRepresentation.feature_task_to_net[task]
torchsummary.summary(net, (3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,408
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]           4,096
       BatchNorm2d-6           [-1, 64, 64, 64]             128
              ReLU-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
             ReLU-10           [-1, 64, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]          16,384
      BatchNorm2d-12          [-1, 256, 64, 64]             512
           Conv2d-13          [-1, 256, 64, 64]          16,384
      BatchNorm2d-14          [-1, 256,

## Define Layers to extract activations

In [3]:
_, eval_nodes = get_graph_node_names(net)
return_nodes = {node:node for node in eval_nodes if "conv" in node or "fc" in node}
activation_extractor = create_feature_extractor(net, return_nodes=return_nodes)

In [4]:
data = ImageDatasetTaskonomy(os.path.join('data','stimuli_places1','untransformed','scale8'), beauty_ratings_path='./behavior/ratings_study1.csv')

In [5]:
integration_ratings = calculate_dataset_integration(data, activation_extractor)

/home/max/miniconda3/envs/PytorchEnv/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [6]:
integration_ratings

,conv1,layer1.0.conv1,layer1.0.conv2,layer1.0.conv3,layer1.1.conv1,layer1.1.conv2,layer1.1.conv3,layer1.2.conv1,layer1.2.conv2,layer1.2.conv3,...,layer3.5.conv3,layer4.0.conv1,layer4.0.conv2,layer4.0.conv3,layer4.1.conv1,layer4.1.conv2,layer4.1.conv3,layer4.2.conv1,layer4.2.conv2,layer4.2.conv3
0,-0.999991,-0.936642,-0.934466,-0.935969,-0.904005,-0.935814,-0.931759,-0.918483,-0.911972,-0.890295,...,-0.137930,-0.770316,-0.453550,-0.456116,-0.594658,-0.359765,-0.178945,-0.347093,-0.281436,-0.135682
1,-0.999955,-0.963097,-0.884371,-0.890870,-0.906541,-0.925186,-0.910882,-0.925319,-0.903241,-0.878101,...,-0.920808,-0.990737,-0.896368,-0.936358,-0.929399,-0.823115,-0.915983,-0.912726,-0.839880,-0.905891
2,-0.999948,-0.959299,-0.862562,-0.866453,-0.903425,-0.888381,-0.885857,-0.918011,-0.901663,-0.846007,...,-0.887986,-0.991643,-0.874940,-0.924446,-0.919121,-0.789649,-0.890146,-0.913709,-0.841277,-0.894605
3,-0.999968,-0.952015,-0.918668,-0.916533,-0.912042,-0.931676,-0.923608,-0.928480,-0.914450,-0.888074,...,-0.214854,-0.879364,-0.609108,-0.485275,-0.752628,-0.663677,-0.454453,-0.683164,-0.776617,-0.865294
4,-0.999962,-0.942360,-0.892700,-0.889042,-0.909401,-0.898860,-0.882733,-0.924383,-0.911258,-0.859058,...,-0.867012,-0.988206,-0.849169,-0.868761,-0.908572,-0.765086,-0.820054,-0.886940,-0.779974,-0.780867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,-0.999974,-0.956059,-0.908246,-0.908545,-0.915434,-0.922431,-0.911444,-0.918918,-0.906877,-0.870916,...,-0.890650,-0.986374,-0.857403,-0.919065,-0.909058,-0.791113,-0.905559,-0.905908,-0.812710,-0.891740
246,-0.999966,-0.955644,-0.913804,-0.911800,-0.918820,-0.929592,-0.919092,-0.925997,-0.915954,-0.884163,...,-0.903080,-0.988678,-0.885641,-0.932794,-0.918998,-0.812912,-0.903828,-0.915996,-0.859072,-0.912142
247,-0.999982,-0.927003,-0.916459,-0.919193,-0.912465,-0.925605,-0.920740,-0.923770,-0.927946,-0.893598,...,-0.849464,-0.977486,-0.751189,-0.886827,-0.880077,-0.743810,-0.886731,-0.904454,-0.809486,-0.885961
248,-0.999916,-0.955343,-0.757361,-0.765798,-0.886407,-0.864660,-0.830860,-0.922551,-0.877884,-0.838867,...,-0.820735,-0.992869,-0.855935,-0.845368,-0.928452,-0.700598,-0.822742,-0.909202,-0.789041,-0.807226


In [10]:
from scipy.stats import spearmanr

In [20]:
def correlate_integration_beauty(integration_ratings: pd.DataFrame, beauty_ratings: pd.DataFrame):
    return integration_ratings.aggregate(lambda x: spearmanr(x, beauty_ratings)[0], axis= 0)

In [31]:
correlate_integration_beauty(integration_ratings, data.beauty_ratings)

conv1            -0.109754
layer1.0.conv1    0.107449
layer1.0.conv2   -0.022921
layer1.0.conv3   -0.041183
layer1.1.conv1   -0.014913
layer1.1.conv2   -0.045467
layer1.1.conv3   -0.023069
layer1.2.conv1   -0.030365
layer1.2.conv2    0.019254
layer1.2.conv3   -0.014621
layer2.0.conv1   -0.024087
layer2.0.conv2   -0.123377
layer2.0.conv3   -0.085724
layer2.1.conv1   -0.023309
layer2.1.conv2    0.008717
layer2.1.conv3    0.025263
layer2.2.conv1    0.102302
layer2.2.conv2    0.194538
layer2.2.conv3    0.082341
layer2.3.conv1    0.084602
layer2.3.conv2    0.119133
layer2.3.conv3    0.149158
layer3.0.conv1    0.023309
layer3.0.conv2    0.078263
layer3.0.conv3   -0.038633
layer3.1.conv1    0.083620
layer3.1.conv2    0.004144
layer3.1.conv3    0.042955
layer3.2.conv1    0.025898
layer3.2.conv2    0.014242
layer3.2.conv3   -0.076855
layer3.3.conv1    0.033383
layer3.3.conv2    0.038585
layer3.3.conv3         NaN
layer3.4.conv1    0.031699
layer3.4.conv2    0.029416
layer3.4.conv3   -0.037299
l

# Inspect

In [3]:
class ImageDataset(object):
    """
    Handles preparing images for input into activation extractors:
        
        - Load images (matlab arrays) from subfolder,
            in alphanumerical order (corresponding to beauty ratings in file).
        
        - Transform into PyTorch format
    
    This class provides a iterator to do so.
    """
    def __init__(self, img_dir, beauty_ratings_path=None):

        dir_img_list    = list(f for f in os.listdir(os.path.join(img_dir, 'full')))
        self.img_dir    = img_dir
        self.img_list   = sorted(dir_img_list)
        self.img_count  = len(dir_img_list)
        if beauty_ratings_path is not None:
            self.beauty_ratings = pd.read_csv(beauty_ratings_path, header=None).mean(axis=1)

    def __iter__(self):
        self.img_pos = 0
        return self
    
    def __next__(self):
        if self.img_pos < self.img_count:
            # load arrays (transformed in matlab)
            img_full = loadmat(os.path.join(self.img_dir,'full', self.img_list[self.img_pos]))["im"]
            img_v1 = loadmat(os.path.join(self.img_dir,'version1', self.img_list[self.img_pos]))["imv1"]
            img_v2 = loadmat(os.path.join(self.img_dir,'version2', self.img_list[self.img_pos]))["imv2"]
            
            # convert to input format of Taskonomy models
            img_full = torch.tensor(img_full).permute([2, 0, 1]).unsqueeze(0)
            img_v1 = torch.tensor(img_v1).permute([2, 0, 1]).unsqueeze(0)
            img_v2 = torch.tensor(img_v2).permute([2, 0, 1]).unsqueeze(0)
            self.img_pos += 1
            return img_full, img_v1, img_v2
        else: # prepare for a possible next iteration
            self.img_pos = 0
            raise StopIteration

In [26]:
# --- Correlation (-Integration)
def correlation_coeff(net, img_full, img_v1, img_v2):
    """Calculate correlation coefficient between full and average activation pattern"""    
    # activations for full image and image parts
    with torch.no_grad():
        act_full, act_v1, act_v2 = net(img_full), net(img_v1), net(img_v2)

    integration = {}
    for (layer, act_full_, act_v1_, act_v2_) in zip(act_full.keys(), act_full.values(), act_v1.values(), act_v2.values()):
        # average activation for image parts
        act_avg_ = torch.stack((act_v1_, act_v2_), dim=0).mean(dim=0).flatten()
        act_full_ = act_full_.flatten()
        
        integration[layer] = pearsonr(act_full_, act_avg_)[0]
        if isnan(integration[layer]):
            
            globals().update({'af': act_full_, 'aa': act_avg_})
            sys.exit("Error message")
        

    return integration


def calculate_dataset_correlation(ImageDataset_iterator, net):
    """Calculate integration for whole dataset"""
    lst = []
    for img_full, img_v1, img_v2 in ImageDataset_iterator:
        lst.append(correlation_coeff(net, img_full, img_v1, img_v2))
    
    column_names = list(net(torch.zeros(1,3,256,256)).keys())
    return pd.DataFrame(lst, columns=column_names) 

In [5]:
model_name = 'autoencoding'

DATASET_NAMES = ('places1', 'places2', 'oasis')
SCALE_NAMES = ('scale2','scale4','scale8','scale16','scale32')

DATA_PATH = './data_256x256'
BEHAVIOR_PATH = './behavior'
RESULTS_PATH = './results_taskonomy'

In [27]:
# Import taskonomy model...
VisualPriorRepresentation._load_unloaded_nets([model_name])
net = VisualPriorRepresentation.feature_task_to_net[model_name]

# ...and create activation extractor from it
_, eval_nodes = get_graph_node_names(net)
return_nodes = { node:node for node in eval_nodes if "conv" in node or 'fc' in node}
activation_extractor = create_feature_extractor(net, return_nodes=return_nodes)

dataset_name =  DATASET_NAMES[0]

for scale_name in SCALE_NAMES:
    
    dataset = ImageDataset(
        os.path.join(DATA_PATH, dataset_name, 'untransformed', scale_name))
                
    correlations = calculate_dataset_correlation(dataset, activation_extractor)
    correlations.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'correlations.csv'), index=False, header=False)


    #selfsimilarity = classes.calculate_dataset_self_similarity(dataset, activation_extractor)
    #selfsimilarity.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'selfsimilarity.csv'), index=False, header=False)

    #l2norm = classes.calculate_dataset_l2norm(dataset, activation_extractor)
    #l2norm.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'l2norm.csv'), index=False, header=False)

    break


/home/max/miniconda3/envs/PytorchEnv/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


SystemExit: Error message

/home/max/miniconda3/envs/PytorchEnv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [28]:
af

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [29]:
aa

tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
         0.0000e+00, -1.6382e-08])